In [4]:
import numpy as np
import torch 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
# Stoke vector for the input polarized light

'''
     z    |
          |
          |________ x
      y  /
'''


s_in = np.array([[1,0,0,0]],dtype="float")

s_in_d =  torch.tensor(s_in,device=device)

print(s_in_d.size())

# Polarizer 
P_in = 0.5*np.array([[1,0,0,1],
                     [0,0,0,0],
                     [0,0,0,0],
                     [1,0,0,1]])
P_in_d =  torch.tensor(P_in,device=device)   

#theoretical pure twist result 
def director(x,z,lam):


     return (np.pi*x/lam

# Calculate Mueller matrix for the jth particle



# Analyzer 
P_out = 0.5*np.array([[1,0,0,-1],
                      [0,0,0,0],
                      [0,0,0,0],
                      [-1,0,0,1]])
P_out_d =  torch.tensor(P_out,device=device)    


s_out_d = P_out_d @ P_in_d @ s_in_d.T

print(s_out_d)

cuda:0
torch.Size([1, 4])
tensor([[0.],
        [0.],
        [0.],
        [0.]], device='cuda:0', dtype=torch.float64)
